# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [23]:
import time
import os
import math

import numpy as np

def cool_argmin(array):
    return np.unravel_index(np.argmin(array), array.shape)

In [7]:
def read_tsp_instance(path: str) -> list:
    with open(path, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

def euclidean_distance(point1: tuple, point2: tuple) -> float:
    if len(point1) != len(point2):
        raise ValueError("Points must have equal dimensions")
    
    distance = 0
    for j in range(len(point1)):
        distance += (point1[j] - point2[j]) ** 2
    return math.sqrt(distance)

def calculate_tour_length(instance, permutation):
    assert(len(instance) == len(permutation))
    
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [20]:
# input -- [(x1, y1), (x2, y2)]
# output -- nearest neighbour algo vertices sequence
def solve_tsp_nearest_neighbour(instance: list):
    n = len(instance)
    def w(i, j):
        return euclidean_distance(instance[i], instance[j])
    
    not_visited = np.array(range(1, n))
    answer = np.array([0])
    
    for i in range(1, n):
        distances = np.array([w(answer[-1], not_visited[j]) for j in range(n - i)])
        min_i = np.argmin(distances)
        answer = np.insert(answer, i, not_visited[min_i])
        not_visited = np.delete(not_visited, min_i)
    
    return answer

In [112]:
def solve_tsp_nearest_insertion(instance):
    instance = np.array([list(pt) for pt in instance])
    
    n = len(instance)
    if n == 1:
        return [0]
    
    def w(i, j):
        return euclidean_distance(instance[i], instance[j])
    
    cycle = np.array([0])
    not_visited = np.array(range(1, n))
    
    for cycle_len in range(1, n):
        in_cycle = instance[cycle].T
        not_in_cycle = instance[not_visited].T
        
        A1 = np.tile(in_cycle, n - cycle_len).reshape((2, n - cycle_len, cycle_len))
        A1 = np.array([x.T for x in A1])
        
        B1 = np.tile(not_in_cycle, cycle_len).reshape((2, cycle_len, n - cycle_len))
        D = np.sqrt((A1[0] - B1[0]) ** 2 + (A1[1] - B1[1]) ** 2)
        dist_v_u1_v_u2 = D + np.roll(D, 1, axis=0)
        neigh_dist_in_cycle = np.array([w(cycle[i - 1], cycle[i]) for i in range(cycle_len)])
        M = dist_v_u1_v_u2.T - neigh_dist_in_cycle
        
        optimal = np.argmin(M)
        cycle_edge_i, i_not_in_cycle = optimal % cycle_len, optimal // cycle_len
        
        cycle = np.insert(cycle, cycle_edge_i, not_visited[i_not_in_cycle])
        not_visited = np.delete(not_visited, i_not_in_cycle)
    
    assert(len(cycle) == n)
    return cycle
        

In [115]:
def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        path = 'tsp-instances/{file}'.format(file=filename)
        if not os.path.exists(path):
            print('File not found: “{}”. Skipping this instance.'.format(path))
            continue
        instance = read_tsp_instance(path)
        print('Solving instance {}…\n'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done \n NN: {:.2} seconds, Tour length {} \n NI: {:.2} seconds, Tour length {}'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [116]:
run_all()

Solving instance d198.tsp…
 done 
 NN: 0.058 seconds, Tour length 18620 
 NI: 0.085 seconds, Tour length 17631
Solving instance d493.tsp…
 done 
 NN: 0.19 seconds, Tour length 43646 
 NI: 0.75 seconds, Tour length 39982
Solving instance d657.tsp…
 done 
 NN: 0.34 seconds, Tour length 62176 
 NI: 1.7 seconds, Tour length 57906
Solving instance d2103.tsp…
 done 
 NN: 3.0 seconds, Tour length 87468 
 NI: 6.9e+01 seconds, Tour length 87570
Solving instance pr107.tsp…
 done 
 NN: 0.012 seconds, Tour length 46678 
 NI: 0.028 seconds, Tour length 52587
Solving instance pr152.tsp…
 done 
 NN: 0.021 seconds, Tour length 85702 
 NI: 0.051 seconds, Tour length 88530
Solving instance pr439.tsp…
 done 
 NN: 0.15 seconds, Tour length 131282 
 NI: 0.51 seconds, Tour length 130067


## Выводы
Как видно по результатам запусков, нельзя заранее сказать, какой алгоритм сработает лучше, однако в большинстве случаев Nearest Insertion работает лучше, но цена тому -- более сложная асимптотика -- $O(n^3)$, вместо $O(n^2)$ у Nearest Neightbour